In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Binarizer
from sklearn.compose import make_column_transformer

# ORIG 

In [22]:
headers_orig = open('headers_orig.txt', 'r')
headers_orig = headers_orig.read().strip().split('\n')
orig = pd.read_csv("sample_orig_2018.txt", index_col=False, names=headers_orig, sep="|")
orig.head()

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,784,201803,9,204802,49180.0,0,1,I,75,39,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
2,731,201803,9,204802,NaN,0,1,P,45,37,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
3,763,201802,Y,204801,NaN,30,1,P,95,34,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,755,201803,9,204802,49700.0,0,1,P,80,33,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N


# SVCG

In [18]:
headers_svcg = open('headers_svcg.txt', 'r')
headers_svcg = headers_svcg.read().strip().split('\n')
svcg = pd.read_csv("sample_svcg_2018.txt", index_col=False, names=headers_svcg, sep="|")
svcg.head()

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Repurchase Flag,Modification Flag,Zero Balance Code,Zero Balance Effective Date,...,Miscellaneous Expenses,Actual Loss Calculation,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code
0,F18Q10000027,201802,74000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN
1,F18Q10000027,201803,74000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN
2,F18Q10000027,201804,73000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN
3,F18Q10000027,201805,73000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN
4,F18Q10000027,201806,73000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN


# Join SVCG+Orig -> new_sample.csv

In [25]:
new_svcg = svcg.head(50000)
new_svcg = new_svcg.join(orig.set_index("Loan Sequence Number"), on="Loan Sequence Number", how="left")
new_svcg.to_csv('new_sample.csv', index=False) 

In [4]:
# zero_balance_codes = {1: 0, 2: 0, 3: 1, 6: 1, 9: 1, 15: 0, np.nan:0}
# svcg['Zero Balance Code'] = svcg['Zero Balance Code'].replace(zero_balance_codes)
# svcg['Current Loan Delinquency Status'] = svcg['Current Loan Delinquency Status'].replace({'R': 100, 'XX': 100}).astype(int)

In [5]:
# bi_1 = Binarizer(threshold=2)
# ct_1 = make_column_transformer(
#     (bi, ['Current Loan Delinquency Status']),
#     remainder='passthrough'
# )

# new_svcg = pd.DataFrame(ct.fit_transform(svcg))
# new_svcg

In [6]:
def current_loan_helper(val):g
            return 1
        else:
            return 0
    except:
        return 1
    
def zero_balance_helper(val):
    target = [3,6,9]
    if val in target:
        return 1
    else:
        return 0
    
def default_helper(val):
    if val > 0:
        return 1
    else:
        return 0

In [7]:
svcg['Zero Balance Code'] = svcg['Zero Balance Code'].apply(zero_balance_helper)

In [8]:
svcg['Current Loan Delinquency Status'] = svcg['Current Loan Delinquency Status'].apply(current_loan_helper)

In [9]:
svcg = svcg.assign(**{'Default': svcg['Current Loan Delinquency Status'] + svcg['Zero Balance Code']})
svcg['Default'] = svcg['Default'].apply(default_helper)